In [1]:
import kagglehub
import pandas as pd
import numpy as np
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

2025-06-05 18:26:13.994290: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749137174.054325  292436 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749137174.072058  292436 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749137174.172605  292436 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749137174.172640  292436 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749137174.172645  292436 computation_placer.cc:177] computation placer alr

In [2]:
# Download latest version
path = kagglehub.dataset_download("danofer/sarcasm")

print("Path to dataset files:", path)

Path to dataset files: /home/eugen/.cache/kagglehub/datasets/danofer/sarcasm/versions/4


In [3]:
dataset = pd.read_csv(path + '/train-balanced-sarcasm.csv')
dataset.head(3)

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.


In [ ]:
dataset = dataset[['label', 'comment']]
dataset = dataset.sample(n=10000)
dataset.info(), dataset.head(3), dataset.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 50000 entries, 479311 to 203967
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    50000 non-null  int64 
 1   comment  49994 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


(None,
         label                                            comment
 479311      0  Or just use LVOC, it's also not banable and yo...
 340977      1           Do you want to have your rent increased?
 294911      0                        on EA Sports golf maybe....,
 label      0
 comment    6
 dtype: int64)

In [5]:
dataset.dropna(inplace=True)
dataset.isna().sum()

label      0
comment    0
dtype: int64

In [6]:
dataset['comment'] = dataset['comment'].apply(lambda comment: comment.lower())
dataset['comment'] = dataset['comment'].str.replace(r'[^a-zA-Z\s]', '', regex=True)
dataset.sample(10)

,label,comment
473287,1,but its ok since he has be forgiven by the bea...
81004,0,this training will be done by one of the good one
74205,0,saaaaayy do you want to go see a mooovie
1000390,1,i wonder if the us will veto this as well
446723,0,edmund pls
320268,1,it absolutely does
810533,1,cool whens the neon going underneath
733114,1,you love him yet you cant even spell his name ...
926256,1,sounds like a totally legit reason
846203,1,thats because its better


In [7]:
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')

In [8]:
def tokenize_text(text):
    return tokenizer(
         text.tolist(),
         max_length = 100,
         truncation = True,
         padding = 'max_length',
         return_tensors = 'np'
    )

tokenized_comments = tokenize_text(dataset['comment'])
tokenized_comments

{'input_ids': array([[  101,  2030,  2074, ...,     0,     0,     0],
       [  101,  2079,  2017, ...,     0,     0,     0],
       [  101,  2006, 19413, ...,     0,     0,     0],
       ...,
       [  101,  2833,   102, ...,     0,     0,     0],
       [  101, 15624,   102, ...,     0,     0,     0],
       [  101,  4608,  2100, ...,     0,     0,     0]]), 'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])}

In [9]:
bert_model = TFBertModel.from_pretrained('bert-base-uncased')
model = tf.keras.Sequential([
    bert_model,
    tf.keras.layers.Lambda(lambda bert_output: bert_output.pooler_output, name="extract_pooler_output"),
    tf.keras.layers.Dense(128, activation='relu'),
    #tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu'),
    #tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

I0000 00:00:1749137201.607649  292436 gpu_process_state.cc:208] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1749137201.608085  292436 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6257 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
2025-06-05 18:26:42.417564: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initia

In [ ]:
from sklearn.model_selection import train_test_split


labels_tf = tf.constant(dataset['label'].values)
indices = np.arange(dataset.shape[0])
train_indices, test_indices = train_test_split(
    indices,
    test_size=0.2,
    stratify=dataset['label'].values
)

X_train = {
        'input_ids': tf.gather(tokenized_comments['input_ids'], train_indices),
        'attention_mask': tf.gather(tokenized_comments['attention_mask'], train_indices)
    }

X_test = {
    'input_ids': tf.gather(tokenized_comments['input_ids'], test_indices),
    'attention_mask': tf.gather(tokenized_comments['attention_mask'], test_indices)
}

y_train = tf.gather(labels_tf, train_indices)
y_test = tf.gather(labels_tf, test_indices)

model.fit(X_train, y_train, epochs=10, batch_size=16)

Epoch 1/3


I0000 00:00:1749137226.055056  292546 service.cc:152] XLA service 0x7abe5ec707f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1749137226.055084  292546 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2025-06-05 18:27:06.065899: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1749137226.092621  292546 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1749137226.227456  292546 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2500/2500 [==============================] - 425s 150ms/step - loss: 0.5843 - accuracy: 0.6872
Epoch 2/3
2500/2500 [==============================] - 360s 144ms/step - loss: 0.4743 - accuracy: 0.7749
Epoch 3/3
2500/2500 [==============================] - 360s 144ms/step - loss: 0.3096 - accuracy: 0.8723


In [11]:
y_pred_trained = model.predict(X_train)
y_pred_test = model.predict(X_test)

  85/1250 [=>............................] - ETA: 2:08

KeyboardInterrupt: 

In [ ]:
max(y_pred_trained), min(y_pred_trained), max(y_pred_test), min(y_pred_test)

(array([0.99869955], dtype=float32),
 array([0.01339717], dtype=float32),
 array([0.99862707], dtype=float32),
 array([0.01438333], dtype=float32))

In [ ]:
y_pred_train_results = (y_pred_trained > 0.5).astype(int).flatten()
y_pred_test_results = (y_pred_test > 0.5).astype(int).flatten()

full_pred = np.empty(len(dataset), dtype=int)

full_pred[train_indices] = y_pred_train_results
full_pred[test_indices] = y_pred_test_results
    
dataset['predicted'] = full_pred
dataset.sample(20)

,label,comment,predicted
589228,0,haha no if boston fans hate bart then im expec...,0
507884,1,but dont worry we made up for it with sorry day,1
512443,1,got rejected from dream resume booster plz help,0
528637,1,yeah acne is a really secret brand and the chi...,1
483220,1,positive navi comment fangay that needs to stfu,0
740575,1,shhhh nothing to see here move along people mo...,0
765010,0,sure sounds like it,0
321043,0,i hated being made to feel i was doing somethi...,0
837152,1,is it a picture of koishi,0
94617,0,i wished the cartoon looked actually good inst...,0


In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred_test_results)
print(report)

              precision    recall  f1-score   support

           0       0.67      0.78      0.72      1994
           1       0.74      0.62      0.67      2006

    accuracy                           0.70      4000
   macro avg       0.70      0.70      0.70      4000
weighted avg       0.70      0.70      0.70      4000

